<a id="content"></a>
## Notebook content

This notebook contains a Scikit-learn representation of AutoAI pipeline. This notebook introduces commands for retrieving data, training the model, and testing the model. 

Some familiarity with Python is helpful. This notebook uses Python 3.11 and scikit-learn 1.3.

## Notebook goals

-  Scikit-learn pipeline definition
-  Pipeline training 
-  Pipeline evaluation

## Contents

This notebook contains the following parts:

**[Setup](#setup)**<br>
&nbsp;&nbsp;[Package installation](#install)<br>
&nbsp;&nbsp;[AutoAI experiment metadata](#variables_definition)<br>
&nbsp;&nbsp;[Watson Machine Learning connection](#connection)<br>
**[Pipeline inspection](#inspection)** <br>
&nbsp;&nbsp;[Read training data](#read)<br>
&nbsp;&nbsp;[Train and test data split](#split)<br>
&nbsp;&nbsp;[Create pipeline](#preview_model_to_python_code)<br>
&nbsp;&nbsp;[Train pipeline model](#train)<br>
&nbsp;&nbsp;[Test pipeline model](#test_model)<br>
**[Store the model](#saving)**<br>
**[Summary and next steps](#summary_and_next_steps)**<br>
**[Copyrights](#copyrights)**

<a id="setup"></a>
# Setup

<a id="install"></a>
## Package installation
Before you use the sample code in this notebook, install the following packages:
 - ibm-watsonx-ai,
 - autoai-libs,
 - scikit-learn,
 - xgboost


Filter warnings for this notebook.

In [75]:
import numpy as np
import pandas as pd
from sklearn.pipeline import make_pipeline, make_union
from sklearn.preprocessing import FunctionTransformer
from sklearn.impute import SimpleImputer
from sklearn.decomposition import PCA
from xgboost import XGBClassifier

In [76]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score, StratifiedKFold

<a id="variables_definition"></a>
## AutoAI experiment metadata
The following cell contains the training data connection details.  
**Note**: The connection might contain authorization credentials, so be careful when sharing the notebook.

The following cell contains input parameters provided to run the AutoAI experiment in Watson Studio.

In [77]:
def read_csv_file(file_path):
    """
    Reads a CSV file and returns its content as a pandas DataFrame.
    
    Parameters:
    file_path (str): The path to the CSV file.
    
    Returns:
    pd.DataFrame: DataFrame containing the CSV data.
    """
    try:
        df = pd.read_csv(file_path, )
        return df
    except ValueError as e:
        print(f"Error reading CSV file: {e}")
        return None
    
train_data = pd.read_csv('data/train_data_80.csv')
descriptors_train = train_data.iloc[:, 1:200]
labels_train = train_data.iloc[:, -1]

test_data = pd.read_csv('data/train_data_20.csv')
descriptors_test = test_data.iloc[:, 1:200]
labels_test = test_data.iloc[:, -1]
smiles_test = test_data.iloc[:, 0]


<a id="inspection"></a>
# Pipeline inspection

<a id="read"></a>
## Read training data

Retrieve training dataset from AutoAI experiment as pandas DataFrame.

**Note**: If reading data results in an error, provide data as Pandas DataFrame object, for example, reading .CSV file with `pandas.read_csv()`.

It may be necessary to use methods for initial data pre-processing like: e.g. `DataFrame.dropna()`, `DataFrame.drop_duplicates()`, `DataFrame.sample()`.


In [78]:

X_train, X_test, y_train, y_test = descriptors_train, descriptors_test, labels_train, labels_test

<a id="preview_model_to_python_code"></a>
## Create pipeline
In the next cell, you can find the Scikit-learn definition of the selected AutoAI pipeline.

#### Import statements.

#### Pre-processing & Estimator.

In [79]:
xgb_classifier = XGBClassifier(
    gamma=1,
    learning_rate=0.07109801185110007,
    max_depth=6,
    min_child_weight=2,
    missing=float("nan"),
    n_estimators=468,
    random_state=42,
    reg_alpha=0,
    reg_lambda=1.0,
    subsample=0.9627910629945398,
    tree_method="hist",
    verbosity=0,
    silent=True,
    colsample_bytree=0.8,
)



#### Pipeline.

In [80]:
def cohen_kappa(annotator1, annotator2):
    # Créer la matrice de confusion
    confusion_matrix = np.zeros((max(annotator1 + annotator2), max(annotator1 + annotator2)))
    for a1, a2 in zip(annotator1, annotator2):
        confusion_matrix[a1-1][a2-1] += 1

    # Calculer p_o
    po = np.trace(confusion_matrix) / len(annotator1)

    # Calculer p_e
    row_sums = np.sum(confusion_matrix, axis=1)
    col_sums = np.sum(confusion_matrix, axis=0)
    pe = np.sum(row_sums * col_sums) / (len(annotator1) ** 2)

    # Calculer Kappa
    kappa = (po - pe) / (1 - pe)

    return kappa




<a id="train"></a>
## Train pipeline model


### Define scorer from the optimization metric
This cell constructs the cell scorer based on the experiment metadata.

<a id="test_model"></a>
### Fit pipeline model
In this cell, the pipeline is fitted.

In [81]:
xgb_classifier.fit(X_train.values, y_train.values)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.8, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              feature_weights=None, gamma=1, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.07109801185110007, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=6, max_leaves=None,
              min_child_weight=2, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=468, n_jobs=None,
              num_parallel_tree=None, ...)

In [82]:
y_pred = xgb_classifier.predict(X_test.values)
kappa_score = cohen_kappa(y_test.values, y_pred)
print(f"Cohen's Kappa Score: {kappa_score}")
print(f"Accuracy: {xgb_classifier.score(X_test.values, y_test.values)}")

Cohen's Kappa Score: 0.6208778286922805
Accuracy: 0.8104089219330854


<a id="saving"></a>
## Store the model

In this section you will learn how to store the trained model.

In [83]:
from joblib import dump

# Enregistrer le modèle entraîné
# dump(xgb_classifier, 'models/xgb_classifier_model.joblib')

Inspect the stored model details.

In [84]:
def save_predictions(smiles_list, y_pred, name):
    """
    Save predictions to a CSV file.
    
    Parameters:
    smiles_list (list): List of SMILES strings.
    y_pred (list): List of predicted labels.
    output_path (str): Path to save the predictions CSV file.
    """
    if len(smiles_list) != len(y_pred):
        raise ValueError("Length of SMILES list and predictions must match.")
    
    output_path='predictions/' + name + '.csv'
    
    # Create a DataFrame and save it to CSV 
    df = pd.DataFrame({
        "smiles": smiles_list,
        name : y_pred
    })
    df.to_csv(output_path, index=False)
    print(f"✅ Prédictions enregistrées dans : {output_path}")

def save_probas(smiles_list, y_proba, name):
    """
    Save predictions to a CSV file.
    
    Parameters:
    smiles_list (list): List of SMILES strings.
    y_pred (list): List of predicted labels.
    output_path (str): Path to save the predictions CSV file.
    """
    if len(smiles_list) != len(y_proba):
        raise ValueError("Length of SMILES list and predictions must match.")
    
    output_path='predictions/' + name + '.csv'

    # Create a DataFrame and save it to CSV 
    df = pd.DataFrame({
        "smiles": smiles_list,
        name : y_proba
    })
    df.to_csv(output_path, index=False)
    print(f"✅ Prédictions enregistrées dans : {output_path}")






save_predictions(smiles_test, y_pred, 'xgb_pred')
y_proba = xgb_classifier.predict_proba(X_test)
save_probas(smiles_test, y_proba[:, 1], 'xgb_proba')

✅ Prédictions enregistrées dans : predictions/xgb_pred.csv
✅ Prédictions enregistrées dans : predictions/xgb_proba.csv


In [85]:
# Entrainer le classificateur XGBoost avec la validation croisée
xgb_classifier.fit(X.values, y.values)

# Définir la validation croisée stratifiée
stratified_kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

# Effectuer la validation croisée stratifiée
scores = cross_val_score(xgb_classifier, X.values, y.values, cv=stratified_kfold)
# print(f"Cross-Validation Scores: {scores}")
print(f"Mean Cross-Validation Score: {np.mean(scores)}")
print(f"Standard Deviation of Cross-Validation Scores: {np.std(scores)}")

Mean Cross-Validation Score: 0.8161450189638796
Standard Deviation of Cross-Validation Scores: 0.012757764565035476


# TEST 1

In [91]:
X_test_1_raw = pd.read_csv('data/test_1.csv')
X_test_1 = X_test_1_raw.iloc[:, 1:200]
y_proba_1 = xgb_classifier.predict_proba(X_test_1.values)
save_probas(X_test_1_raw.iloc[:, 0].tolist(), y_proba_1[:, 1], 'xgb_proba_test_1')

✅ Prédictions enregistrées dans : predictions/xgb_proba_test_1.csv
